# Proiect de Analiză a Recenziilor Spotify
## Student: Popa Anda-Ioana
### Grupa: 408

# Cuprins
1. [Introducere](#introducere)
2. [Implementarea de script-uri Spark pentru procesarea datelor](#implementarea)
3. [Aplicarea a cel puțin două metode ML](#aplicarea)
4. [Utilizarea unui Data Pipeline](#utilizarea)

# 1. Introducere <a name="introducere"></a>

## a. Prezentarea succintă a setului de date

Setul de date ales pentru acest proiect este intitulat **Spotify Reviews** și poate fi descărcat de pe Kaggle, accesând următorul link: [aici](https://www.kaggle.com/datasets/ashishkumarak/spotify-reviews-playstore-daily-update?resource=download&select=spotify_reviews.csv). Acesta conține recenzii ale utilizatorilor pentru aplicația Spotify, preluate de pe Google Play Store. Datele sunt actualizate zilnic și includ diverse atribute relevante pentru evaluarea feedback-ului utilizatorilor.

Structura setului de date include următoarele coloane principale:
- **reviewId**: Un identificator unic pentru fiecare recenzie.
- **userName**: Numele utilizatorului care a scris recenzia.
- **userImage**: URL-ul imaginii de profil a utilizatorului.
- **content**: Textul recenziei.
- **score**: Scorul oferit aplicației (de la 1 la 5 stele).
- **thumbsUpCount**: Numărul de utilizatori care au găsit recenzia utilă.
- **reviewCreatedVersion**: Versiunea aplicației la momentul recenziei.
- **at**: Data la care a fost scrisă recenzia.
- **replyContent**: Răspunsul dezvoltatorului la recenzie (dacă există).
- **repliedAt**: Data la care dezvoltatorul a răspuns recenziei.

Acest set de date este valoros pentru analizarea părerii utilizatorilor față de aplicația Spotify, identificarea principalelor probleme întâmpinate de utilizatori, și evaluarea impactului actualizărilor aplicației asupra experienței utilizatorilor.

## b. Enunțarea obiectivelor

Scopul acestui proiect este de a efectua o analiză cuprinzătoare a recenziilor utilizatorilor pentru aplicația Spotify, folosind Apache Spark pentru procesarea și transformarea datelor, precum și metode de învățare automată pentru clasificarea impresiilor și alte analize relevante. Proiectul va fi structurat în mai multe etape, fiecare având obiective specifice, după cum urmează:

### 1. Preprocesarea și Curățarea Datelor:
- Curățarea textului recenziilor pentru a elimina caracterele speciale și zgomotul.
- Tratarea valorilor lipsă și a datelor eronate.
- Conversia datelor într-un format adecvat pentru analize ulterioare.

### 2. Explorarea și Vizualizarea Datelor:
- Explorarea statistică a setului de date pentru a înțelege distribuția scorurilor, frecvența recenziilor pe diverse versiuni ale aplicației, și alte aspecte relevante.
- Vizualizarea datelor folosind grafice și diagrame pentru a identifica tendințele și modelele emergente.

### 3. Analiza Părerii:
- Utilizarea Spark MLlib pentru a aplica metode de învățare automată, cum ar fi clasificarea sentimentelor (pozitiv, negativ, neutru) în recenziile utilizatorilor.
- Evaluarea performanței modelelor utilizate și interpretarea rezultatelor obținute.

### 4. Identificarea Temelor Principale:
- Aplicarea tehnicilor de grupare și agregare pentru a identifica temele principale și problemele recurente menționate în recenzii.
- Utilizarea Spark SQL și DataFrames pentru a efectua aceste analize.

### 5. Crearea unui Data Pipeline:
- Implementarea unui pipeline de date pentru a automatiza procesul de preprocesare, analiză și clasificare a recenziilor.
- Asigurarea scalabilității și eficienței pipeline-ului pentru a putea gestiona volume mari de date.

Prin atingerea acestor obiective, proiectul va oferi o perspectivă detaliată asupra feedback-ului utilizatorilor pentru aplicația Spotify, identificând aspectele pozitive și negative și oferind recomandări pentru îmbunătățirea experienței utilizatorilor.


# 2. Implementarea de script-uri Spark pentru procesarea datelor <a name="implementarea"></a>

Pentru acest proiect, voi folosi Apache Spark pentru a prelucra, curăța și transforma datele. Vom utiliza DataFrames și Spark SQL pentru a efectua operații de grupare și agregare. 

## 2.1 Configurarea Mediului Spark

Prima etapă este configurarea mediului Spark și încărcarea setului de date.


In [2]:
# Importarea bibliotecilor necesare 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, regexp_replace, to_timestamp, avg, count, desc 

# Inițializarea unei sesiuni Spark 
spark = SparkSession.builder \
    .appName("Spotify Reviews Analysis") \
    .getOrCreate() 

# Încărcarea setului de date 
file_path = "D:\Master anul 1 sem 2\Big Data\spotify_reviews.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Vizualizarea primelor rânduri ale DataFrame-ului 
df.show(5)


+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+----------+
|            reviewId|            userName|             content|score|thumbsUpCount|reviewCreatedVersion|                 at|appVersion|
+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+----------+
|437314fe-1b1d-435...|           Rajib Das|           It's good|    4|            0|                NULL|2024-05-09 16:28:13|      NULL|
|4933ad2c-c70a-4a8...|Mihaela Claudia N...|I love this app s...|    5|            0|          8.9.38.494|2024-05-09 16:27:18|8.9.38.494|
|1ab275fb-59bf-42c...|     JONATHAN GRACIA|             Perfect|    5|            0|          8.9.36.616|2024-05-09 16:27:03|8.9.36.616|
|b38406eb-7b11-4ce...|          Cam Rempel|Best all around m...|    5|            0|          8.9.38.494|2024-05-09 16:26:19|8.9.38.494|
|7be7999d-4cb6-47b...|Your clowness (He..

## 2.2 Preprocesarea și Curățarea Datelor
Pentru preprocesarea datelor, voi curăța textul recenziilor, voi trata valorile lipsă și voi converti coloanele la tipurile de date adecvate.


In [3]:
# Eliminarea valorilor lipsă 
df_clean = df.na.drop(subset=["content", "score", "at"]) 
# Curățarea textului recenziilor pentru a elimina caracterele speciale 
df_clean = df_clean.withColumn("clean_content", regexp_replace(col("content"), "[^a-zA-Z0-9\s]", "")) 
# Conversia coloanei 'at' la tipul de date Data 
df_clean = df_clean.withColumn("review_date", to_timestamp(col("at"), "yyyy-MM-dd HH:mm:ss")) 
# Vizualizarea primelor rânduri după curățare 
df_clean.show(5)

+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+----------+--------------------+-------------------+
|            reviewId|            userName|             content|score|thumbsUpCount|reviewCreatedVersion|                 at|appVersion|       clean_content|        review_date|
+--------------------+--------------------+--------------------+-----+-------------+--------------------+-------------------+----------+--------------------+-------------------+
|437314fe-1b1d-435...|           Rajib Das|           It's good|    4|            0|                NULL|2024-05-09 16:28:13|      NULL|            Its good|2024-05-09 16:28:13|
|4933ad2c-c70a-4a8...|Mihaela Claudia N...|I love this app s...|    5|            0|          8.9.38.494|2024-05-09 16:27:18|8.9.38.494|I love this app s...|2024-05-09 16:27:18|
|1ab275fb-59bf-42c...|     JONATHAN GRACIA|             Perfect|    5|            0|          8.9.36.616|2024-

## 2.3 Explorarea și Vizualizarea Datelor
Voi explora setul de date pentru a înțelege distribuția scorurilor și alte statistici descriptive.


In [4]:
# Calcularea distribuției scorurilor 
score_distribution = df_clean.groupBy("score").count().orderBy("score") 
# Afișarea distribuției scorurilor 
score_distribution.show() 
# Calcularea scorului mediu pentru fiecare versiune a aplicației 
average_score_per_version = df_clean.groupBy("reviewCreatedVersion").agg(avg("score").alias("average_score")) 
# Afișarea scorului mediu pe versiuni 
average_score_per_version.orderBy(desc("average_score")).show()

+--------------------+-----+
|               score|count|
+--------------------+-----+
|         ""Artists""|    1|
|     ""Dismiss"" and|    1|
| ""Events"" etc. ...|    1|
| ""I know what wi...|    1|
| ""Recently Liste...|    1|
| ""Spotify offline""|    1|
|          ""albums""|    1|
| ""listen to two ...|    1|
| ""music"" can't ...|    1|
|    ""new playlist""|    1|
| ""no user found ...|    1|
|         ""ooh ahh""|    1|
|      ""the pocket""|    1|
| ""wait"" or ""se...|    1|
| ""wven"" of I br...|    1|
|          $10 a YEAR|    1|
| & can't play oth...|    1|
| & everything els...|    1|
| & now it's stuck...|    1|
| (which will how ...|    1|
+--------------------+-----+
only showing top 20 rows

+--------------------+------------------+
|reviewCreatedVersion|     average_score|
+--------------------+------------------+
|           8.9.8.417|               5.0|
|          8.4.72.842|               5.0|
|          8.4.36.321|               5.0|
|          8.5.11.582|    

## 2.4 Analiza Sentimentului
Voi aplica metode de învățare automată pentru a clasifica sentimentele recenziilor. În această etapă, voi pregăti datele pentru modelare.


In [5]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF 
from pyspark.ml.classification import LogisticRegression 
from pyspark.ml import Pipeline 
# Tokenizarea textului recenziilor 
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")
# Eliminarea cuvintelor comune (stop words)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Convertirea textului în vectori de frecvență a termenilor 
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
# Calcularea TF-IDF 
idf = IDF(inputCol="raw_features", outputCol="features")
# Configurarea etichetei pentru clasificare
df_clean = df_clean.withColumn("label", (col("score") >= 4).cast("int"))
# Eliminarea valorilor lipsă sau NaN în coloana "label" și "clean_content"
df_clean = df_clean.na.drop(subset=["label", "clean_content"])
# Crearea modelului de regresie logistică
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Crearea pipeline-ului
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])
# Împărțirea setului de date în date de antrenament și date de test
(trainingData, testData) = df_clean.randomSplit([0.8, 0.2], seed=1234)
# Antrenarea modelului
model = pipeline.fit(trainingData)
# Evaluarea modelului pe datele de test
predictions = model.transform(testData)
# Afișarea rezultatelor
predictions.select("clean_content", "probability", "prediction", "label").show(5)

+--------------------+--------------------+----------+-----+
|       clean_content|         probability|prediction|label|
+--------------------+--------------------+----------+-----+
|I love using spot...|[0.11314725935005...|       1.0|    1|
|I love my premium...|[0.23074400965238...|       1.0|    1|
|The fact that the...|[0.98632583844751...|       0.0|    0|
|Huge issue with t...|[0.15691094324704...|       1.0|    1|
|After several upd...|[0.71191757889934...|       0.0|    0|
+--------------------+--------------------+----------+-----+
only showing top 5 rows



## 2.5 Grupări și Agregări de Date
Voi efectua operații de grupare și agregare pentru a identifica principalele teme și probleme din recenzii.


In [6]:
# Gruparea recenziilor pe scoruri și calcularea numărului de recenzii utile 
useful_reviews = df_clean.groupBy("score").agg(count("thumbsUpCount").alias("useful_reviews")) 
# Afișarea recenziilor utile pe scoruri 
useful_reviews.orderBy(desc("useful_reviews")).show() 
# Gruparea recenziilor pe versiuni și calcularea scorului mediu 
version_score = df_clean.groupBy("reviewCreatedVersion").agg(avg("score").alias("average_score")) 
# Afișarea scorului mediu pe versiuni 
version_score.orderBy(desc("average_score")).show()

+-----+--------------+
|score|useful_reviews|
+-----+--------------+
|    1|         33213|
|    5|         13256|
|    2|         12902|
|    3|         11322|
|    4|          9230|
+-----+--------------+

+--------------------+------------------+
|reviewCreatedVersion|     average_score|
+--------------------+------------------+
|           8.9.8.417|               5.0|
|          8.4.72.842|               5.0|
|          8.4.36.321|               5.0|
|          8.5.11.582|               5.0|
|          8.4.35.152|               4.5|
|           3.3.0.988|               4.0|
|          8.4.72.845| 3.878651685393258|
|         8.4.85.1006|3.8666666666666667|
|          8.9.36.616| 3.720576873551378|
|          8.9.38.494| 3.686725663716814|
|          8.4.74.463|3.6346153846153846|
|          8.4.81.558|3.5348837209302326|
|          8.9.24.632|               3.5|
|          8.8.94.566|               3.5|
|           8.3.0.681|               3.5|
|          8.6.52.916|              

# 3. Aplicarea a cel puțin două metode ML <a name="aplicarea"></a>

În această secțiune, voi aplica două metode de învățare automată folosind Spark MLlib: clasificarea sentimentului recenziilor și regresia pentru a prezice numărul de recenzii utile. Voi descrie problema, justificarea alegerii metodei, explicarea soluției și evaluarea fiecărei metode.

## 3.1 Clasificarea Sentimentului Recenziilor

### a. Enunțul problemei

Problema este de a clasifica recenziile utilizatorilor pentru aplicația Spotify ca fiind pozitive sau negative pe baza scorului oferit. Voi considera recenziile cu scoruri de 4 și 5 ca fiind pozitive și cele cu scoruri de 1, 2 și 3 ca fiind negative.

### Justificarea alegerii metodei

Am ales metoda de clasificare logistică deoarece este eficientă pentru problemele binare și poate oferi o bună interpretabilitate a rezultatelor. Logistic Regression este bine adaptată pentru această problemă datorită naturii binare a etichetei noastre (pozitiv/negativ).

### Explicarea soluției

Voi folosi Spark MLlib pentru a construi un pipeline de procesare a textului și pentru a antrena un model de clasificare logistică. Pipeline-ul va include tokenizarea, eliminarea cuvintelor comune (stop words), conversia textului în vectori de frecvență a termenilor (TF) și calcularea TF-IDF, urmate de aplicarea regresiei logistice.

### b. Aplicarea și evaluarea metodei


In [7]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
# Tokenizarea textului recenziilor
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")
# Eliminarea cuvintelor comune (stop words)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Convertirea textului în vectori de frecvență a termenilor
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
# Calcularea TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")
# Configurarea etichetei pentru clasificare
df_clean = df_clean.withColumn("label", (col("score") >= 4).cast("int"))
# Crearea modelului de regresie logistică
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Crearea pipeline-ului
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, lr])
# Împărțirea setului de date în date de antrenament și date de test
(trainingData, testData) = df_clean.randomSplit([0.8, 0.2], seed=1234)
# Antrenarea modelului
model = pipeline.fit(trainingData)
# Evaluarea modelului pe datele de test
predictions = model.transform(testData)
evaluator = BinaryClassificationEvaluator()
# Calcularea AUC (area under the curve)
auc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})
print(f"Area Under ROC Curve (AUC): {auc}")
# Afișarea primelor predicții
predictions.select("clean_content", "probability", "prediction", "label").show(5)

Area Under ROC Curve (AUC): 0.8854976201483787
+--------------------+--------------------+----------+-----+
|       clean_content|         probability|prediction|label|
+--------------------+--------------------+----------+-----+
|I love using spot...|[0.11314725935005...|       1.0|    1|
|I love my premium...|[0.23074400965238...|       1.0|    1|
|The fact that the...|[0.98632583844751...|       0.0|    0|
|Huge issue with t...|[0.15691094324704...|       1.0|    1|
|After several upd...|[0.71191757889934...|       0.0|    0|
+--------------------+--------------------+----------+-----+
only showing top 5 rows



### Evaluarea metodei
AUC (area under the ROC curve) este o măsură utilizată pentru a evalua performanța modelului de clasificare. Un AUC apropiat de 1 indică un model bun, în timp ce un AUC apropiat de 0.5 indică un model care nu este mai bun decât ghicitul aleatoriu metodei


## 3.2 Predicția Numărului de Recenzii Utile (Regresie)
### a. Enunțul problemei
Problema este de a prezice numărul de recenzii utile (thumbsUpCount) pe baza conținutului recenziilor și a altor atribute. Aceasta este o problemă de regresie, deoarece trebuie să prezicem o valoare numerică continuă.
### Justificarea alegerii metodei
Am ales metoda de regresie liniară deoarece este simplă și eficientă pentru problemele de regresie. Linear Regression poate oferi o interpretare directă a relației dintre caracteristicile de intrare și valoarea de ieșire.
### Explicarea soluției
Voi folosi Spark MLlib pentru a construi un pipeline de procesare a textului și pentru a antrena un model de regresie liniară. Pipeline-ul va include aceleași etape de preprocesare a textului (tokenizare, eliminare stop words, TF-IDF) urmate de aplicarea regresiei liniare.
### b. Aplicarea și evaluarea metodei

In [8]:
from pyspark.sql.functions import col, regexp_replace, when, length
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# Curățarea valorilor non-numerice din coloana "thumbsUpCount"
df_clean = df_clean.withColumn("thumbsUpCount", regexp_replace(col("thumbsUpCount"), "[^0-9]", ""))

# Înlocuirea șirurilor de caractere goale cu '0'
df_clean = df_clean.withColumn("thumbsUpCount", when(col("thumbsUpCount") == "", "0").otherwise(col("thumbsUpCount")))

# Convertirea coloanei "thumbsUpCount" la tip numeric
df_clean = df_clean.withColumn("thumbsUpCount", col("thumbsUpCount").cast("int"))

# Eliminarea valorilor lipsă sau NaN în coloana "thumbsUpCount" și "clean_content"
df_clean = df_clean.na.drop(subset=["thumbsUpCount", "clean_content"])

# Adăugarea lungimii recenziei ca o caracteristică suplimentară
df_clean = df_clean.withColumn("review_length", length(col("clean_content")))

# Tokenizarea textului recenziilor 
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")

# Eliminarea cuvintelor comune (stop words)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Convertirea textului în vectori de frecvență a termenilor 
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)

# Calcularea TF-IDF 
idf = IDF(inputCol="raw_features", outputCol="features")

# Vectorizarea caracteristicilor (inclusiv lungimea recenziei)
assembler = VectorAssembler(inputCols=["features", "review_length"], outputCol="final_features")

# Crearea modelului de regresie liniară
lr_regression = LinearRegression(featuresCol="final_features", labelCol="thumbsUpCount")

# Crearea pipeline-ului
pipeline_regression = Pipeline(stages=[tokenizer, remover, hashingTF, idf, assembler, lr_regression])

# Împărțirea setului de date în date de antrenament și date de test
(trainingData, testData) = df_clean.randomSplit([0.8, 0.2], seed=1234)

# Antrenarea modelului
model_regression = pipeline_regression.fit(trainingData)

# Evaluarea modelului pe datele de test
predictions_regression = model_regression.transform(testData)

# Evaluarea modelului folosind RMSE (root mean squared error)
evaluator_regression = RegressionEvaluator(labelCol="thumbsUpCount", predictionCol="prediction", metricName="rmse")
rmse = evaluator_regression.evaluate(predictions_regression)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Afișarea primelor predicții
predictions_regression.select("clean_content", "thumbsUpCount", "prediction").show(5)


Root Mean Squared Error (RMSE): 122.3070248228727
+--------------------+-------------+-------------------+
|       clean_content|thumbsUpCount|         prediction|
+--------------------+-------------+-------------------+
|I love using spot...|            0|-5.7457375333182465|
|I love my premium...|            0|  -8.12011514820541|
|The fact that the...|           13| 44.790924910877195|
|Huge issue with t...|            2| -7.359868963862047|
|After several upd...|            0|  12.35054337764135|
+--------------------+-------------+-------------------+
only showing top 5 rows



### Evaluarea metodei
RMSE (root mean squared error) este o măsură utilizată pentru a evalua performanța unui model de regresie. Un RMSE mai mic indică un model mai bun, deoarece erorile de predicție sunt mai mici.


## 3.3 Gruparea Recenziilor Utilizatorilor folosind K-means Clustering
### a. Enunțul Problemei
Problema constă în identificarea grupurilor de recenzii similare bazate pe conținutul acestora. Scopul este de a descoperi teme și modele recurente în recenziile utilizatorilor pentru aplicația Spotify, fără a dispune de etichete predefinite (clustering nesupravegheat).

### Justificarea Alegerii Metodei
Am optat pentru K-means clustering datorită eficienței și popularității sale în rezolvarea problemelor de clustering nesupravegheat. Algoritmul K-means este adecvat pentru identificarea subgrupurilor în seturi mari de date și poate facilita descoperirea de noi perspective și teme în recenziile utilizatorilor.

### Explicarea Soluției
Voi utiliza Spark MLlib pentru a construi un pipeline de preprocesare a textului și pentru a aplica algoritmul K-means. Pipeline-ul va include etapele de tokenizare, eliminare a cuvintelor comune (stop words), conversie a textului în vectori de frecvență a termenilor (TF), calcularea TF-IDF, urmate de aplicarea algoritmului K-means pentru gruparea recenziilor.

### b. Aplicarea și Evaluarea Metodei

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
# Tokenizarea textului recenziilor 
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")
# Eliminarea cuvintelor comune (stop words)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Convertirea textului în vectori de frecvență a termenilor 
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
# Calcularea TF-IDF 
idf = IDF(inputCol="raw_features", outputCol="features")
# Aplicarea K-means clustering
kmeans = KMeans(featuresCol="features", k=5, seed=1234)  # k este numărul de clustere
# Crearea pipeline-ului
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, kmeans])
# Antrenarea modelului
model = pipeline.fit(df_clean)
# Aplicarea modelului pentru a obține predicțiile
predictions = model.transform(df_clean)
# Evaluarea clusteringului folosind Silhouette Score
evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette")
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette}")
# Afișarea primelor predicții
predictions.select("clean_content", "prediction").show(10)

Silhouette Score: 0.0628982526916374
+--------------------+----------+
|       clean_content|prediction|
+--------------------+----------+
|            Its good|         0|
|I love this app s...|         1|
|             Perfect|         0|
|Best all around m...|         0|
|Are yall fr gatek...|         0|
|            Loved it|         0|
|The app is good b...|         0|
|       excellent app|         0|
| I like this spotify|         0|
|        Eat the rich|         0|
+--------------------+----------+
only showing top 10 rows



### Evaluarea Metodei
1) Silhouette Score: Un scor Silhouette mai mare indică o mai bună separare între clustere. Acesta variază între -1 și 1, unde valori mai mari sunt mai bune.
2) Vizualizarea Clusterele: Putem folosi vizualizări (de exemplu, t-SNE sau PCA) pentru a vizualiza cum au fost grupate recenziile în funcție de conținut.

# 4. Utilizarea unui Data Pipeline <a name="utilizarea"></a>

Un Data Pipeline în Apache Spark permite automatizarea proceselor de preprocesare, transformare și aplicare a modelelor de învățare automată. În acest proiect, voi crea un pipeline care include toate etapele necesare pentru clasificarea sentimentului recenziilor Spotify.


Pipeline-ul este creat și utilizat în acest cod pentru a automatiza întregul proces de preprocesare a textului și aplicarea algoritmului 
K-means clustering.

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace, to_date
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
# Tokenizarea textului recenziilor 
tokenizer = Tokenizer(inputCol="clean_content", outputCol="words")
# Eliminarea cuvintelor comune (stop words)
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# Convertirea textului în vectori de frecvență a termenilor 
hashingTF = HashingTF(inputCol="filtered_words", outputCol="raw_features", numFeatures=10000)
# Calcularea TF-IDF 
idf = IDF(inputCol="raw_features", outputCol="features")
# Aplicarea K-means clustering
kmeans = KMeans(featuresCol="features", k=5, seed=1234)  # k este numărul de clustere
# Crearea pipeline-ului
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, kmeans])
# Antrenarea modelului
model = pipeline.fit(df_clean)
# Aplicarea modelului pentru a obține predicțiile
predictions = model.transform(df_clean)
# Evaluarea clusteringului folosind Silhouette Score
evaluator = ClusteringEvaluator(featuresCol="features", metricName="silhouette")
silhouette = evaluator.evaluate(predictions)
print(f"Silhouette Score: {silhouette}")
# Afișarea primelor predicții
predictions.select("clean_content", "prediction").show(10)

Silhouette Score: 0.0628982526916374
+--------------------+----------+
|       clean_content|prediction|
+--------------------+----------+
|            Its good|         0|
|I love this app s...|         1|
|             Perfect|         0|
|Best all around m...|         0|
|Are yall fr gatek...|         0|
|            Loved it|         0|
|The app is good b...|         0|
|       excellent app|         0|
| I like this spotify|         0|
|        Eat the rich|         0|
+--------------------+----------+
only showing top 10 rows

